<a href="https://colab.research.google.com/github/lucarenz1997/recommender_systems/blob/main/Hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hybrid

In [22]:
from google.colab import drive
drive.mount('/content/drive')

# Loading the dataset
import pandas as pd
train_df = pd.read_csv("/content/drive/MyDrive/Recommender/preprocessed_train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Recommender/preprocessed_test.csv")


import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=831160 sha256=e5bbc87364c3803a9a104678b6d7b4fefdf9818c3ae4711f86a0578e4d41e88f
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm


In [13]:
import pandas as pd
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset
from scipy.sparse import csr_matrix

# Ensure LightFM uses only encoded and normalized data
df = train_df.copy()

# Create LightFM Dataset
dataset = Dataset()

# Fit dataset to map users and items to integer IDs
dataset.fit(
    users=df['user_id'].unique(),
    items=df['media_id'].unique()
)

# Build the interaction matrix (implicit feedback)
(interactions, weights) = dataset.build_interactions(
    [(row['user_id'], row['media_id']) for _, row in df.iterrows()]
)

# Print dataset size
num_users, num_items = interactions.shape
print(f"Dataset contains {num_users} users and {num_items} items.")


Dataset contains 13692 users and 61075 items.


In [14]:
# Convert train/test datasets to LightFM format
train_interactions, _ = dataset.build_interactions(
    [(row['user_id'], row['media_id']) for _, row in train_df.iterrows()]
)

# Keep only test items that exist in train_df
valid_test_items = set(train_df['media_id'].unique())  # Get seen items
filtered_test_df = test_df[test_df['media_id'].isin(valid_test_items)]  # Remove unseen items

# Keep only test users that exist in train_df
valid_test_users = set(train_df['user_id'].unique())  # Get seen users
filtered_test_df = filtered_test_df[filtered_test_df['user_id'].isin(valid_test_users)]  # Remove unseen users


# Convert test_df into LightFM format
test_interactions, _ = dataset.build_interactions(
    [(row['user_id'], row['media_id']) for _, row in filtered_test_df.iterrows()]
)

print(f"Filtered test dataset: {len(filtered_test_df)} interactions.")


Filtered test dataset: 48496 interactions.


In [15]:
# Initialize LightFM model
model = LightFM(loss='warp')  # warp = good for implicit feedback

# Train the model
model.fit(train_interactions, epochs=10, num_threads=4)


In [16]:
from lightfm.evaluation import precision_at_k

# Evaluate precision@10
train_precision = precision_at_k(model, train_interactions, k=10).mean()
test_precision = precision_at_k(model, test_interactions, k=10).mean()

print(f"Train Precision@10: {train_precision:.4f}")
print(f"Test Precision@10: {test_precision:.4f}")


Train Precision@10: 0.0600
Test Precision@10: 0.0185


In [17]:
def recommend_songs(model, user_id, dataset, num_recommendations=5):
    """ Recommend songs for a given user using a trained LightFM model """
    n_users, n_items = dataset.interactions_shape()
    scores = model.predict(user_id, np.arange(n_items))  # Predict scores
    top_items = np.argsort(-scores)[:num_recommendations]  # Get top N items

    # Convert back to media_id
    item_mapping = dict(zip(dataset.mapping()[2], dataset.mapping()[1]))
    recommendations = [item_mapping[item] for item in top_items]

    return recommendations

# Example recommendation for user 5
sample_user = 5
recommended_songs = recommend_songs(model, sample_user, dataset)
print(f"Recommended songs for user {sample_user}: {recommended_songs}")


KeyError: 462

In [18]:
import pandas as pd
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k
from scipy.sparse import csr_matrix

# Ensure LightFM uses only encoded and normalized data
df = train_df.copy()

# Create LightFM Dataset
dataset = Dataset()

# Fit dataset to map users and items to integer IDs (ONLY FROM TRAINING DATA)
dataset.fit(
    users=df['user_id'].unique(),
    items=df['media_id'].unique()
)

# Build the interaction matrix (implicit feedback)
(interactions, weights) = dataset.build_interactions(
    [(row['user_id'], row['media_id']) for _, row in df.iterrows()]
)

# Print dataset size
num_users, num_items = interactions.shape
print(f"Dataset contains {num_users} users and {num_items} items.")

# Convert train/test datasets to LightFM format
train_interactions, _ = dataset.build_interactions(
    [(row['user_id'], row['media_id']) for _, row in train_df.iterrows()]
)

# Keep only test items that exist in train_df
valid_test_items = set(train_df['media_id'].unique())  # Get seen items
filtered_test_df = test_df[test_df['media_id'].isin(valid_test_items)]  # Remove unseen items

# Keep only test users that exist in train_df
valid_test_users = set(train_df['user_id'].unique())  # Get seen users
filtered_test_df = filtered_test_df[filtered_test_df['user_id'].isin(valid_test_users)]  # Remove unseen users

# Convert test_df into LightFM format
test_interactions, _ = dataset.build_interactions(
    [(row['user_id'], row['media_id']) for _, row in filtered_test_df.iterrows()]
)

print(f"Filtered test dataset: {len(filtered_test_df)} interactions.")

# Function to train & evaluate LightFM with different hyperparameters
def train_lightfm(train_interactions, test_interactions, loss, no_components, learning_rate, epochs):
    model = LightFM(loss=loss, no_components=no_components, learning_rate=learning_rate)
    model.fit(train_interactions, epochs=epochs, num_threads=4)

    # Evaluate Precision@10
    train_precision = precision_at_k(model, train_interactions, k=10).mean()
    test_precision = precision_at_k(model, test_interactions, k=10).mean()

    return model, train_precision, test_precision

# Hyperparameter tuning options
hyperparams = [
    {'loss': 'warp', 'no_components': 32, 'learning_rate': 0.05, 'epochs': 15},
    {'loss': 'warp', 'no_components': 64, 'learning_rate': 0.01, 'epochs': 20},
    {'loss': 'warp', 'no_components': 128, 'learning_rate': 0.05, 'epochs': 25},
    {'loss': 'bpr', 'no_components': 64, 'learning_rate': 0.01, 'epochs': 15},
    {'loss': 'bpr', 'no_components': 128, 'learning_rate': 0.05, 'epochs': 20}
]

# Run hyperparameter tuning
results = []
for params in hyperparams:
    model, train_prec, test_prec = train_lightfm(
        train_interactions, test_interactions,
        loss=params['loss'],
        no_components=params['no_components'],
        learning_rate=params['learning_rate'],
        epochs=params['epochs']
    )
    results.append((params, train_prec, test_prec))

# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=['Hyperparameters', 'Train Precision@10', 'Test Precision@10'])


Dataset contains 13692 users and 61075 items.
Filtered test dataset: 48496 interactions.


In [24]:
import pandas as pd
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k
from scipy.sparse import csr_matrix

# Ensure LightFM uses only encoded and normalized data
df = train_df.copy()

# Filter out inactive users (users with fewer than 10 interactions)
active_users = df['user_id'].value_counts()[df['user_id'].value_counts() > 20].index
filtered_train_df = df[df['user_id'].isin(active_users)]

# Convert user and item features to string format
filtered_train_df['user_age'] = filtered_train_df['user_age'].astype(str)
filtered_train_df['user_gender'] = filtered_train_df['user_gender'].astype(str)
filtered_train_df['genre_id'] = filtered_train_df['genre_id'].astype(str)
filtered_train_df['song_popularity_7d'] = filtered_train_df['song_popularity_7d'].astype(str)

# Create LightFM Dataset
dataset = Dataset()

# Fit dataset with user & item metadata
dataset.fit(
    users=filtered_train_df['user_id'].unique(),
    items=filtered_train_df['media_id'].unique(),
    user_features=[f"age:{age} gender:{gender}" for age, gender in zip(filtered_train_df['user_age'], filtered_train_df['user_gender'])],
    item_features=[f"genre:{genre} popularity:{pop}" for genre, pop in zip(filtered_train_df['genre_id'], filtered_train_df['song_popularity_7d'])]
)

# Build the interaction matrix (implicit feedback)
train_interactions, _ = dataset.build_interactions(
    [(row['user_id'], row['media_id']) for _, row in filtered_train_df.iterrows()]
)

# Keep only test items that exist in train_df
valid_test_items = set(filtered_train_df['media_id'].unique())  # Get seen items
filtered_test_df = test_df[test_df['media_id'].isin(valid_test_items)]  # Remove unseen items

# Keep only test users that exist in train_df
valid_test_users = set(filtered_train_df['user_id'].unique())  # Get seen users
filtered_test_df = filtered_test_df[filtered_test_df['user_id'].isin(valid_test_users)]  # Remove unseen users

# Convert test_df into LightFM format
test_interactions, _ = dataset.build_interactions(
    [(row['user_id'], row['media_id']) for _, row in filtered_test_df.iterrows()]
)

print(f"Filtered test dataset: {len(filtered_test_df)} interactions.")

# Function to train & evaluate LightFM with specified hyperparameters
def train_lightfm(train_interactions, test_interactions, loss, no_components, learning_rate, epochs, item_alpha, user_alpha):
    model = LightFM(loss=loss, no_components=no_components, learning_rate=learning_rate, item_alpha=item_alpha, user_alpha=user_alpha)
    model.fit(train_interactions, epochs=epochs, num_threads=4)

    # Evaluate Precision@10
    train_precision = precision_at_k(model, train_interactions, k=10).mean()
    test_precision = precision_at_k(model, test_interactions, k=10).mean()

    return model, train_precision, test_precision

# Train LightFM model with optimized hyperparameters
selected_params = {'loss': 'warp', 'no_components': 128, 'learning_rate': 0.01, 'epochs': 50, 'item_alpha': 1e-5, 'user_alpha': 1e-5}
model, train_prec, test_prec = train_lightfm(
    train_interactions, test_interactions,
    loss=selected_params['loss'],
    no_components=selected_params['no_components'],
    learning_rate=selected_params['learning_rate'],
    epochs=selected_params['epochs'],
    item_alpha=selected_params['item_alpha'],
    user_alpha=selected_params['user_alpha']
)

# Display results
print(f"Train Precision@10: {train_prec:.4f}")
print(f"Test Precision@10: {test_prec:.4f}")


Filtered test dataset: 31769 interactions.
Train Precision@10: 0.1701
Test Precision@10: 0.0305


In [19]:
results_df

,Hyperparameters,Train Precision@10,Test Precision@10
0,"{'loss': 'warp', 'no_components': 32, 'learnin...",0.123649,0.022376
1,"{'loss': 'warp', 'no_components': 64, 'learnin...",0.057501,0.018609
2,"{'loss': 'warp', 'no_components': 128, 'learni...",0.481661,0.029070
3,"{'loss': 'bpr', 'no_components': 64, 'learning...",0.018858,0.004591
4,"{'loss': 'bpr', 'no_components': 128, 'learnin...",0.187014,0.020116
